In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense, TimeDistributed, Flatten, Dropout, Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential, load_model
from keras.callbacks import ModelCheckpoint
from numpy import array, argmax
from numpy.random import rand, shuffle
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu
import scipy
import statsmodels
import sklearn
import tensorflow
import keras
from io import open
import unicodedata
import random
import math
import time
import os
import time
import re
import sys

import numpy as np
from unicodedata import normalize
from numpy import array
from pickle import dump, load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.layers import RepeatVector, TimeDistributed
from keras.callbacks import ModelCheckpoint

In [ ]:
MAX_LENGTH = 100


In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1iljWhk1LVNEdXOYyfxCZd_Osv9D5TJ76' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1iljWhk1LVNEdXOYyfxCZd_Osv9D5TJ76" -O assign4dataset.zip && rm -rf /tmp/cookies.txt
! unzip assign4dataset.zip

--2021-04-17 15:49:58--  https://docs.google.com/uc?export=download&confirm=&id=1iljWhk1LVNEdXOYyfxCZd_Osv9D5TJ76
Resolving docs.google.com (docs.google.com)... 172.217.164.174, 2607:f8b0:4004:815::200e
Connecting to docs.google.com (docs.google.com)|172.217.164.174|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-1k-docs.googleusercontent.com/docs/securesc/v645tahvv65b74ld578qjta4nm7j1l8l/3q0sojiov4c74t05h5j3ig3s9gmlk86q/1618674600000/15072518060054569691/08413793949905811032Z/1iljWhk1LVNEdXOYyfxCZd_Osv9D5TJ76?e=download [following]
--2021-04-17 15:50:01--  https://doc-0g-1k-docs.googleusercontent.com/docs/securesc/v645tahvv65b74ld578qjta4nm7j1l8l/3q0sojiov4c74t05h5j3ig3s9gmlk86q/1618674600000/15072518060054569691/08413793949905811032Z/1iljWhk1LVNEdXOYyfxCZd_Osv9D5TJ76?e=download
Resolving doc-0g-1k-docs.googleusercontent.com (doc-0g-1k-docs.googleusercontent.com)... 142.250.73.193, 2607:f8b0:4004:829::2001
Connecting to doc-0g-

In [ ]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {"sos":0, "eos": 1}
        self.word2count = {}
        self.index2word = {0: "sos", 1: "eos"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
def load_doc(filename):
    file = open(filename, mode='rt', encoding='utf-8')
    text = file.read()
    file.close()
    return text

def to_pairs(english_text, hindi_text):
    english_lines = english_text.strip().split('\n')
    hindi_lines = hindi_text.strip().split('\n')
    pairs = []
    for i in range(len(hindi_lines)):
        pairs.append([])
        pairs[i].append(pre_process_english_sentence(english_lines[i]))
        pairs[i].append(pre_process_hindi_sentence(hindi_lines[i]))
    return pairs

In [ ]:
def clean_text(text):
    text = text.replace(u',','')
    text = text.replace(u'"','')
    text = text.replace(u'"','')
    text = text.replace(u"‘‘",'')
    text = text.replace(u"’’",'')
    text = text.replace(u"''",'')
    text = text.replace(u"।",'')
    text=text.replace(u',','')
    text=text.replace(u'"','')
    text=text.replace(u'(','')
    text=text.replace(u')','')
    text=text.replace(u'"','')
    text=text.replace(u':','')
    text=text.replace(u"'",'')
    text=text.replace(u"‘‘",'')
    text=text.replace(u"’’",'')
    text=text.replace(u"''",'')
    text=text.replace(u".",'')
    text=text.replace(u"-",'')
    text=text.replace(u"।",'')
    text=text.replace(u"?",'')
    text=text.replace(u"\\",'')
    text=text.replace(u"_",'')
    text=text.replace("'", "")
    text=text.replace('"', "")
    text= re.sub("'", '', text)
    text= re.sub("’", '', text)
    text=re.sub('[0-9+\-*/.%]', '', text)
    text=text.strip()
    text=re.sub(' +', ' ',text)
    exclude = set(string.punctuation)
    text= ''.join(ch for ch in text if ch not in exclude)
    return text

In [ ]:
def pre_process_english_sentence(line):
    line = line.lower()
    line = clean_text(line)
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    line = normalize('NFD', line).encode('ascii', 'ignore')
    line = line.decode('UTF-8')
    line = line.split()
    line = [re_print.sub('', w) for w in line]
    line = [word for word in line if word.isalpha()]
    line = ' '.join(line)
    return line

def pre_process_hindi_sentence(line):
    line=re.sub('[a-zA-Z]', '', line)
    line = clean_text(line)
    exclude = set(string.punctuation)
    remove_digits = str.maketrans('', '', string.digits)
    line = re.sub("'", '', line)
    line = ''.join(ch for ch in line if ch not in exclude)
    line = line.translate(remove_digits)
    line = re.sub("[२३०८१५७९४६]", "", line)
    line = line.strip()
    line = re.sub(" +", " ", line)
    return (line)

In [ ]:
def prepareData(pairs):
    english_lang = Lang('eng')
    hindi_lang = Lang('hin')
    for pair in pairs:
        english_lang.addSentence(pair[0])
        hindi_lang.addSentence(pair[1])
    return english_lang, hindi_lang

In [ ]:
train_eng_text = load_doc('english.train.txt')
train_hindi_text = load_doc('hindi.train.txt')
train_pair = to_pairs(train_eng_text, train_hindi_text)
english_lang, hindi_lang = prepareData(train_pair)

train_df = pd.DataFrame(train_pair)
train_df.columns = ["english", "hindi"]

list_len = []
for l in train_df.english:
    list_len.append(len(l.split(' ')))
max_len_eng = np.max(list_len)

list_len = []
for l in train_df.hindi:
    list_len.append(len(l.split(' ')))
max_len_hindi = np.max(list_len)


In [ ]:
train_df

,english,hindi
0,give your application an accessibility workout,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
1,accerciser accessibility explorer,एक्सेर्साइसर पहुंचनीयता अन्वेषक
2,the default plugin layout for the bottom panel,निचले पटल के लिए डिफोल्ट प्लगइन खाका
3,the default plugin layout for the top panel,ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका
4,a list of plugins that are disabled by default,उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष...
...,...,...
49995,audio test,ऑडियो जाँच
49996,silent,मूक
49997,video test,वीडियो जाँच
49998,crazy,दीवाना


In [ ]:
test_eng_text = load_doc('english.test.txt')
test_hindi_text = load_doc('hindi.test.txt')
test_pair = to_pairs(test_eng_text, test_hindi_text)
for pair in test_pair:
    english_lang.addSentence(pair[0])
    hindi_lang.addSentence(pair[1])

test_df = pd.DataFrame(test_pair)
test_df.columns = ["english", "hindi"]

list_len = []
for l in test_df.english:
    list_len.append(len(l.split(' ')))
max_len_eng = np.max(list_len)

list_len = []
for l in test_df.hindi:
    list_len.append(len(l.split(' ')))
max_len_hindi = np.max(list_len)



In [ ]:
test_df

,english,hindi
0,a black box in your car,आपकी कार में ब्लैक बॉक्स
1,as americas road planners struggle to find the...,जबकि अमेरिका के सड़क योजनाकार ध्वस्त होते हुए ...
2,the devices which track every mile a motorist ...,यह डिवाइस जो मोटरचालक द्वारा वाहन चलाए गए प्रत...
3,the usually dull arena of highway planning has...,आम तौर पर हाईवे नियोजन जैसा उबाऊ काम भी अचानक ...
4,libertarians have joined environmental groups ...,आपने द्वारा ड्राइव किए गए मील तथा संभवतः ड्राइ...
...,...,...
2502,it is noteworthy that both nita and isha are p...,गौरतलब है कि नीता और ईशा दोनों ही प्रोफेशनल क्...
2503,vips have been invited to this royal party,इस शाही पार्टी के लिए वीवीआईपी लोगों को भी आमं...
2504,these include the jodhpur royal family and uma...,इनमें जोधपुर राजपरिवार और उम्मैद भवन के मालिक ...
2505,ln mittal sachin tendulkar and bollywood actor...,एलएनमित्तल सचिन तेंदुलकर फिल्म अभिनेता अनिल कप...


In [ ]:
# Get Glove Vector
!wget http://nlp.stanford.edu/data/glove.840B.300d.zip
!unzip glove.840B.300d.zip

# using 300-dim Glove word embeddings

glove_embeddings = {}
f = open('glove.840B.300d.txt')
for line in f:
    values = line.split(' ')
    word = values[0] ## The first entry is the word
    embedding = np.asarray(values[1:], dtype='float32') ## These are the vecotrs representing the embedding for the word
    glove_embeddings[word] = embedding 
f.close()

# initializing unknown token (in case a word is not found, we'll assign it an unknown token)
UNK = np.random.random(300)
SOS = np.random.random(300)
EOS = np.random.random(300)

glove_embeddings['sos'] = SOS
glove_embeddings['eos'] = EOS

print('GloVe data loaded')

--2021-04-17 15:50:05--  http://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2021-04-17 15:50:05--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip [following]
--2021-04-17 15:50:06--  http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/zip

In [ ]:
embedding_size = 300
eng_vocab_size = len(english_lang.word2index)+1
embedding_matrix_eng = np.zeros((eng_vocab_size,embedding_size))
for word,i in sorted( english_lang.word2index.items(), key=lambda x: x[1]):
    embedding_value = glove_embeddings.get(word, UNK)
    embedding_matrix_eng[i] = embedding_value

hindi_vocab_size = len(hindi_lang.word2index)+1
embedding_matrix_hindi = np.zeros(( hindi_vocab_size,embedding_size))
for word,i in sorted( hindi_lang.word2index.items(), key=lambda x: x[1]):
    embedding_value = glove_embeddings.get(word, UNK)
    embedding_matrix_hindi[i] = embedding_value

In [ ]:

class EncoderRNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size,n_layers=1, bidirectional=True, batch_size=1):
        super(EncoderRNN, self).__init__()
        self.bidirectional = bidirectional
        self.hidden_size = int(hidden_size/2)
        self.hidden_dim = self.hidden_size
        self.num_layers = n_layers
        self.batch_size = batch_size
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_matrix_eng), freeze=True)
        self.gru = nn.GRU(embedding_size, self.hidden_size, n_layers*2, bidirectional=bidirectional)
        self.linear = nn.Linear(hidden_size * 2, 1)

    def forward(self, input, hidden=None):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        fwd_final = hidden[0:hidden.size(0):2]
        bwd_final = hidden[1:hidden.size(0):2]
        final = torch.cat([fwd_final, bwd_final], dim=2)
        return output, final

    def initHidden(self):
        directions = 2 if self.bidirectional else 1
        return torch.zeros(
            self.num_layers*2,
            self.batch_size,
            self.hidden_size*2,
            device=device
        )

In [ ]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.2, max_length=hindi_lang.n_words,n_layers=1):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p/5
        self.max_length = max_length

        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_matrix_hindi), freeze=True)
        # self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn1 = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size, n_layers*2)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        intmd_torch = torch.cat((embedded[0],hidden[0]),1)
        attn_out= self.attn1(intmd_torch)
        attn_weights = F.softmax(attn_out)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))
       
        output = torch.cat((embedded, attn_applied), 0)
        output = self.attn_combine(output).unsqueeze(0)
        output = F.relu(output)
        output = torch.squeeze(output,0)
        dec_hidden = self.initHidden()
        output1, dec_hidden = self.gru(output, hidden)
        output2 = F.log_softmax(self.out(output1[0]), dim=1)
        return output2, dec_hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1*2, 1, self.hidden_size, device=device)


In [ ]:

def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(english_lang, pair[0])
    target_tensor = tensorFromSentence(hindi_lang, pair[1])
    return (input_tensor, target_tensor)



In [ ]:

teacher_forcing_ratio = 1

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=hindi_lang.n_words):
    
    encoder_hidden = encoder.initHidden()
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    
    encoder_outputs = torch.zeros(max_length, encoder.hidden_dim*2, device=device)
    
    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei])
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    
    decoder_hidden = encoder_hidden
    
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length


In [ ]:


def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    
    random.shuffle(train_pair)
    
    criterion = nn.NLLLoss()
    
    for iter in range(0, len(train_pair)*2 + 1):
        training_pairs = tensorsFromPair(train_pair[(iter)%len(train_pair)])
        training_pair = training_pairs
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        
        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%.4f' % (print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0


In [ ]:


def evaluate(encoder, decoder, sentence, max_length=hindi_lang.n_words):
    with torch.no_grad():
        input_tensor = tensorFromSentence(english_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_outputs = torch.zeros(max_length, encoder.hidden_dim*2, device=device)
        
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei])
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)
        
        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)
        
        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(hindi_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]


In [ ]:


def evaluateRandomly(encoder, decoder, n=20):
    for i in range(n):
        pair = random.choice(test_pair)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')


In [ ]:

embed_dim = 300
hidden_size = embed_dim
hidden_dim = embed_dim
n_layers = 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder1 = EncoderRNN(english_lang.n_words, embed_dim, hidden_dim, n_layers).to(device)
attn_decoder1 = AttnDecoderRNN(embed_dim, hindi_lang.n_words, n_layers).to(device)


In [ ]:
trainIters(encoder1, attn_decoder1, 100, print_every=5000)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0.0019
5.7541
5.4143
5.3271
5.2587
5.2438
5.2121
5.2044
5.1514
5.1453
5.1415
5.1106
5.1342
5.1191
5.0908
5.1046
5.0892
5.0964
5.0580
5.0581
5.0610


In [ ]:

actual, predicted = list(), list()
    
for i in range(len(test_pair)):
    pair = test_pair[i]
    output_words, attentions = evaluate(encoder1, attn_decoder1, pair[0])
    output_sentence = ' '.join(output_words)
    
    hindi_list = pair[1].split()
    predicted_sen = output_sentence.split()
    if (hindi_list[-1] == "<EOS>"):
        hindi_list = hindi_list[:-1]
    if (hindi_list[0] == "<SOS>"):
        hindi_list.pop(0)
    if (predicted_sen[-1] == "<EOS>"):
        predicted_sen = predicted_sen[:-1]
    actual.append(hindi_list)
    predicted.append(predicted_sen)


In [ ]:
print("Bleu Score: ", corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))

Bleu Score:  0.0532591
